# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 14 2023 9:49AM,257326,10,9157787,"Citieffe, Inc.",Released
1,Feb 14 2023 9:36AM,257288,15,VT80184012,"Virtus Pharmaceuticals, LLC",Released
2,Feb 14 2023 9:36AM,257288,15,VT80184013,"Virtus Pharmaceuticals, LLC",Released
3,Feb 14 2023 9:36AM,257288,15,VT80184014,"Virtus Pharmaceuticals, LLC",Released
4,Feb 14 2023 9:36AM,257288,15,VT80184015,"Virtus Pharmaceuticals, LLC",Released
5,Feb 14 2023 9:36AM,257288,15,VT80184016,"Virtus Pharmaceuticals, LLC",Released
6,Feb 14 2023 9:36AM,257288,15,VT80184017,"Virtus Pharmaceuticals, LLC",Released
7,Feb 14 2023 9:36AM,257288,15,VT80184018,"Virtus Pharmaceuticals, LLC",Released
8,Feb 14 2023 9:36AM,257288,15,VT80184019,"Virtus Pharmaceuticals, LLC",Released
9,Feb 14 2023 9:36AM,257288,15,VT80184048,"Virtus Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
45,257320,Released,1
46,257321,Released,1
47,257322,Released,6
48,257323,Released,1
49,257326,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
257320,NaN,1.0
257321,NaN,1.0
257322,NaN,6.0
257323,NaN,1.0
257326,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
257221,0.0,1.0
257231,35.0,28.0
257234,0.0,2.0
257236,0.0,1.0
257238,0.0,11.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
257221,0,1
257231,35,28
257234,0,2
257236,0,1
257238,0,11


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,257221,0,1
1,257231,35,28
2,257234,0,2
3,257236,0,1
4,257238,0,11


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,257221,,1
1,257231,35,28
2,257234,,2
3,257236,,1
4,257238,,11


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 14 2023 9:49AM,257326,10,"Citieffe, Inc."
1,Feb 14 2023 9:36AM,257288,15,"Virtus Pharmaceuticals, LLC"
13,Feb 14 2023 9:36AM,257274,15,"Person & Covey, Inc."
18,Feb 14 2023 9:35AM,257267,15,"Brookfield Pharmaceuticals, LLC"
63,Feb 14 2023 9:35AM,257264,15,"Mizner Bioscience, LLC"
64,Feb 14 2023 9:31AM,257263,15,"Alliance Pharma, Inc."
66,Feb 14 2023 9:29AM,257323,12,SugarBear
67,Feb 14 2023 9:23AM,257322,10,"Digital Brands, LLC"
73,Feb 14 2023 9:21AM,257321,10,Eye Pharma Inc
74,Feb 14 2023 9:20AM,257320,10,"Nextsource Biotechnology, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 14 2023 9:49AM,257326,10,"Citieffe, Inc.",,1
1,Feb 14 2023 9:36AM,257288,15,"Virtus Pharmaceuticals, LLC",,12
2,Feb 14 2023 9:36AM,257274,15,"Person & Covey, Inc.",,5
3,Feb 14 2023 9:35AM,257267,15,"Brookfield Pharmaceuticals, LLC",,45
4,Feb 14 2023 9:35AM,257264,15,"Mizner Bioscience, LLC",,1
5,Feb 14 2023 9:31AM,257263,15,"Alliance Pharma, Inc.",,2
6,Feb 14 2023 9:29AM,257323,12,SugarBear,,1
7,Feb 14 2023 9:23AM,257322,10,"Digital Brands, LLC",,6
8,Feb 14 2023 9:21AM,257321,10,Eye Pharma Inc,,1
9,Feb 14 2023 9:20AM,257320,10,"Nextsource Biotechnology, LLC",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 14 2023 9:49AM,257326,10,"Citieffe, Inc.",1,
1,Feb 14 2023 9:36AM,257288,15,"Virtus Pharmaceuticals, LLC",12,
2,Feb 14 2023 9:36AM,257274,15,"Person & Covey, Inc.",5,
3,Feb 14 2023 9:35AM,257267,15,"Brookfield Pharmaceuticals, LLC",45,
4,Feb 14 2023 9:35AM,257264,15,"Mizner Bioscience, LLC",1,
5,Feb 14 2023 9:31AM,257263,15,"Alliance Pharma, Inc.",2,
6,Feb 14 2023 9:29AM,257323,12,SugarBear,1,
7,Feb 14 2023 9:23AM,257322,10,"Digital Brands, LLC",6,
8,Feb 14 2023 9:21AM,257321,10,Eye Pharma Inc,1,
9,Feb 14 2023 9:20AM,257320,10,"Nextsource Biotechnology, LLC",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 14 2023 9:49AM,257326,10,"Citieffe, Inc.",1,
1,Feb 14 2023 9:36AM,257288,15,"Virtus Pharmaceuticals, LLC",12,
2,Feb 14 2023 9:36AM,257274,15,"Person & Covey, Inc.",5,
3,Feb 14 2023 9:35AM,257267,15,"Brookfield Pharmaceuticals, LLC",45,
4,Feb 14 2023 9:35AM,257264,15,"Mizner Bioscience, LLC",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 14 2023 9:49AM,257326,10,"Citieffe, Inc.",1.0,NaN
1,Feb 14 2023 9:36AM,257288,15,"Virtus Pharmaceuticals, LLC",12.0,NaN
2,Feb 14 2023 9:36AM,257274,15,"Person & Covey, Inc.",5.0,NaN
3,Feb 14 2023 9:35AM,257267,15,"Brookfield Pharmaceuticals, LLC",45.0,NaN
4,Feb 14 2023 9:35AM,257264,15,"Mizner Bioscience, LLC",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 14 2023 9:49AM,257326,10,"Citieffe, Inc.",1.0,0.0
1,Feb 14 2023 9:36AM,257288,15,"Virtus Pharmaceuticals, LLC",12.0,0.0
2,Feb 14 2023 9:36AM,257274,15,"Person & Covey, Inc.",5.0,0.0
3,Feb 14 2023 9:35AM,257267,15,"Brookfield Pharmaceuticals, LLC",45.0,0.0
4,Feb 14 2023 9:35AM,257264,15,"Mizner Bioscience, LLC",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,6689438,264.0,35.0
12,771957,2.0,1.0
15,1286356,65.0,0.0
19,2572882,83.0,5.0
20,514598,4.0,0.0
21,771776,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,6689438,264.0,35.0
1,12,771957,2.0,1.0
2,15,1286356,65.0,0.0
3,19,2572882,83.0,5.0
4,20,514598,4.0,0.0
5,21,771776,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,264.0,35.0
1,12,2.0,1.0
2,15,65.0,0.0
3,19,83.0,5.0
4,20,4.0,0.0
5,21,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,264.0
1,12,Released,2.0
2,15,Released,65.0
3,19,Released,83.0
4,20,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Executing,35.0,1.0,0.0,5.0,0.0,0.0
Released,264.0,2.0,65.0,83.0,4.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Executing,35.0,1.0,0.0,5.0,0.0,0.0
1,Released,264.0,2.0,65.0,83.0,4.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Executing,35.0,1.0,0.0,5.0,0.0,0.0
1,Released,264.0,2.0,65.0,83.0,4.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()